# Ejemplo calcular TF-IDF y similitud coseno

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de mayo de 2025

En ese ejemplo calcularemos la similitud global entre documentos de texto.

```
pip install scikit-learn pandas
```

Para cada archivo de texto se calcula un descriptor global (llamado TF-IDF). Esos descriptores se comparan y para cada documento se obtendrá el documento más parecido del mismo conjunto.

Para este ejemplo se usarán los documentos en `dataset_ciperchile.21-05-2025.zip`. Este dataset contiene 5.174 documentos obtenidos desde el sitio web https://www.ciperchile.cl/ el día 21 de mayo de 2025.

Dataset publicado solo para fines académicos. El derecho de autor de todos los textos pertenece a CIPER Chile https://ciperchile.cl/

El archivo `dataset.txt` lista todos los documentos del dataset. Es un archivo de 6 columnas separadas por el caracter tabulador (\t):
  1. Filename: Nombre del archivo en la carpeta dataset
  2. Titulo: Título del artículo
  3. Por: Autor del artículo
  4. Fecha: Fecha de publicación del artículo
  5. Tipo_documento: Categoría del documento
  6. URL: dirección pública del artículo


La carpeta `dataset` están los textos de cada artículo. Cada archivo de nombre `file00000.txt` tienen la siguiente estructura: 
  * La primera línea es la URL del documento original  
  * La segunda línea es el título del documento  
  * Desde la tercera línea en adelante es el contenido del artículo

Todos los archivos usan codificación UTF-8.

In [ ]:
import time
import numpy
import os

class File():
    def __init__(self):
        self.filename = ""
        self.titulo = ""
        self.autor = ""
        self.fecha = ""
        self.tipo = ""
        self.url = ""
        self.full_text = ""

    def load_full_text(self, dir_dataset):
        file = os.path.join(dir_dataset, self.filename)
        with open(file, mode="r", encoding="utf8") as f:
            self.full_text = f.read()
        # quitar la primera linea (es una url)
        self.full_text = self.full_text[self.full_text.find("\n") + 1 :]
  

def load_dataset(dir_dataset):
    files = list()
    header = dict()
    file_dataset = os.path.join(dir_dataset, 'dataset.txt')
    print("leyendo {}...".format(file_dataset))
    with open(file_dataset, mode="r", encoding="utf8") as f:
        for line in f:
            fields = line.strip().split("\t")
            # leer los nombres de la primera fila 
            if len(header) == 0:
                header = dict((c, i) for i, c in enumerate(fields))
                continue
            file = File()
            file.filename = fields[header["Filename"]]
            file.titulo = fields[header["Titulo"]]
            file.autor = fields[header["Por"]]
            file.fecha = fields[header["Fecha"]]
            file.tipo = fields[header["Tipo_documento"]]
            file.url = fields[header["URL"]]
            files.append(file)
    dir_dataset = os.path.join(dir_dataset, 'dataset')
    print("leyendo textos de {} archivos en {} ...".format(len(files), dir_dataset))
    for file in files:
        file.load_full_text(dir_dataset)
    return files

t1 = time.time()
dataset_files = load_dataset('dataset_ciperchile.21-05-2025')
t2 = time.time()

print("leídos {} documentos en {:.1f} segundos".format(len(dataset_files), t2-t1))

## Opcional, restringir los documentos a usar

A veces es muy lento usar todos los documentos del dataset o puede faltar memoria.

Para reducir la cantidad de documentos a usar cambiar el número a continuación.

In [ ]:
# se podrían usar menos documentos, para que no sea tan lento
cantidad_documentos = 5174  #len(dataset_files)

dataset_files = dataset_files[0:cantidad_documentos]

print("Usando {} documentos".format(len(dataset_files)))

## Calcular vocabulario

Se usa **TfidfVectorizer** para calcular el vocabulario y crear descriptores.  
Ver la documentación en: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#un array con los textos para crear el vocabulario y descriptores
textos = list()
largo_total = 0
for doc in dataset_files:
    largo_total += len(doc.full_text)
    textos.append(doc.full_text)

print("Se van a usar {} documentos, con largo total {:.1f} MB".format(len(textos), largo_total/1024/1024))

#calcular el vocabulario: total de palabras a representar
t0 = time.time()
vectorizer = TfidfVectorizer(lowercase=True,          # por defecto es True 
                             strip_accents='unicode', # por defecto es None. unicode=eliminar acentos
                             sublinear_tf=True,       # por defecto es False. TRUE=usar 1+log(freq) 
                             use_idf=True,            # por defecto es True
                             norm='l2',               # por defecto es l2 (asi el coseno es solo la multiplicacion de valores)
                             ngram_range=(1,1),       # por defecto es (1,1). rango de ngramas a usar por ej: (1,1) o (1,2) o (1,3)
                             max_df=1.0, # Si una palabra aparece en más que max_df documentos, se ignora (probar con 0.9)
                                         #  Si es float -> porcentaje del total de documentos
                                         #  Si es int   -> cantidad de documentos
                                         #  Notar que es distinto 1.0 (el 100% de los documentos) vs 1 (solo 1 documento)
                             min_df=0.0  # Si una palabra aparece en menos que min_df documentos, se ignora (probar con 0.1)
                             )           #  También puede ser float o int
vectorizer.fit(textos)
t1 = time.time()

print("tiempo para crear vocabulario: {:.1f} segs".format(t1-t0))
print("vocabulario de {} palabras".format(len(vectorizer.vocabulary_)))
print("primeras 10 palabras: ", list(vectorizer.vocabulary_.keys())[0:10])
print("primeras 10 palabras ordenadas: ", list(sorted(vectorizer.vocabulary_.keys())[0:10]))

## Calcular descriptores

Se calcula el descriptor td-idf para cada documento.

Notar que `TfidfVectorizer` entrega los descriptores como una matriz de tipo `scipy.sparse._csr.csr_matrix`. Esto implica que ocupa poco espacio en memoria porque solo se guardan los valores distintos de cero.


In [ ]:
#transform() entrega el descriptor tf-idf (un vector sparse con los pesos de las palabras)
t0 = time.time()
descriptores = vectorizer.transform(textos)
t1 = time.time()

print("tiempo descriptores: {:.1f} segs".format(t1-t0))
print("descriptores es matriz de {}".format(descriptores.shape))

#verificar que los descriptores es una matriz sparse
print("descriptores tipo {}".format(type(descriptores)))

# Calcular similares (opción 1: multiplicar matrices)

Calcularemos la "autosimilitud" que consiste en comparar un dataset consigo mismo. Como los descriptores están normalizados L2, al multiplicar la matriz de descriptores con su transpuesta es lo mismo que calcular la similitud coseno entre todos los descriptores.

La matriz de autosimilitud  siempre tendrá 1 en la diagonal porque representa la similitud de un descriptor consigo mismo.

Notar que la matriz es tipo `sparse`, por lo que para poder multiplicar con ella será necesario convertirla en matriz densa, llamando a `toarray()`. Dependiendo de la cantidad de términos del vocabulario esto podría fallar por memoria.


In [ ]:
#usar la funcion toarray() para convertir los descriptores a una matriz densa
#multiplicar las matrices con matmul() para calcular la similitud coseno

t0 = time.time()
descriptores_denso = descriptores.toarray()
auto_similitud = numpy.matmul(descriptores_denso, descriptores_denso.T)
t1 = time.time()

print("Tiempo comparación todos contra todos: {:.1f} segs".format(t1-t0))

auto_similitud

## Imprimir el resultado de similitud coseno

Para cada fila se imprime su más cercano (mayor similitud). Para evitar el valor 1 de la diagonal se escribe 0 en la diagonal.

In [ ]:
import pandas

def imprimir_similares(matriz, min_score):
    #obtener el maximo por fila
    #primero llenar la diagonal con ceros para que el mas parecido no sea si mismo
    numpy.fill_diagonal(matriz, 0)
    
    filas = []
    for i in range(matriz.shape[0]):
        #obtener la posicion del maximo por fila
        posicion_mayor = numpy.argmax(matriz[i])
        #nombres de archivos
        nombre_documento = dataset_files[i].filename
        nombre_mas_similar = dataset_files[posicion_mayor].filename
        similitud = matriz[i, posicion_mayor]
        #imprimir documentos con score mayor al mínimo pedido
        if similitud >= min_score:
            fila = {'id':  i,
                    'query':  nombre_documento,
                    'documento': nombre_mas_similar,
                    'similitud coseno': similitud}
            filas.append(fila)
    
    print("documentos con similitud mayor a {}:".format(min_score))
    print()
    df = pandas.DataFrame(filas)
    print(df.to_string(index=False, justify='center'))

imprimir_similares(auto_similitud, 0)

## Revisar algunos documentos parecidos

Se muestra un documento y su más parecido, para verificar si son similares o no.

In [ ]:
import random

def imprimir_documento(similitudes, i):
    # obtener la posicion del maximo por fila
    posicion_mayor = numpy.argmax(similitudes[i])
    similitud = similitudes[i, posicion_mayor]
    # nombres de archivos
    nombre_documento = dataset_files[i].filename
    url_documento = dataset_files[i].url
    texto_documento = dataset_files[i].full_text
    # el mas parecido
    nombre_similar = dataset_files[posicion_mayor].filename
    url_similar = dataset_files[posicion_mayor].url
    texto_similar = dataset_files[posicion_mayor].full_text
    # mostrar solo el inicio de cada coumento
    max_length = 1000
    if len(texto_documento) > max_length:
        texto_documento = texto_documento[0:max_length]
    if len(texto_similar) > max_length:
        texto_similar = texto_similar[0:max_length]
    # mostrar
    print("Comparando {} y {} con similitud {:.3f}".format(nombre_documento, nombre_similar, similitud))
    print("--------------------------------------")
    print("  ", nombre_documento, " -> ", url_documento)
    print(texto_documento)
    print("--------------------------------------")
    print("--------------------------------------")
    print("  ", nombre_similar, " -> ", url_similar)
    print(texto_similar)
    print("--------------------------------------")

# mostrar algun documento
id_documento = random.randint(0, auto_similitud.shape[0] + 1) 

imprimir_documento(auto_similitud, id_documento)

__Preguntas:__
  * En su opinión, ¿son parecidos los dos documentos mostrados anteriormente?
  * ¿Tiene que ver el nivel parecido con el valor de similitud? (probar con varios, por ejemplo con similitud 0.1 vs similitud 0.4)



# Calcular similares (opción 2: usar cdist)

En vez de multiplicar matrices probaremos la función `cdist()` que ya hemos usado en ejemplos anteriores. Para comparar descriptores configurar la distancia `cosine`, que es igual a (1 - similitud coseno).  
Ver métrica 6 en: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

**Notar la diferencia de tiempo comparado con la multiplicación de matrices!!**   
Usando `cdist()` para comparar descriptores **demora cerca de una hora**, mientras que la multiplicación de matrices toma algunos segundos. En ambos se obtiene exactamente el mismo resultado.

¿A qué se deberá la diferencia en tiempo?

In [ ]:
from scipy.spatial import distance

t0 = time.time()
descriptores_denso = descriptores.toarray()
distancias = distance.cdist(descriptores_denso, descriptores_denso, metric='cosine')
t1 = time.time()

print("tiempo comparacion usando cdist con distancia coseno: {:.1f} segs ({:.1f} mins)".format(t1-t0,(t1-t0)/60))

In [ ]:
import pandas

def evaluar_mas_parecido(matriz_distancias, matriz_ideal):
    #completar la diagonal con un valor muy grande para el mas cercano no sea si mismo
    numpy.fill_diagonal(matriz_distancias, numpy.inf)
    #completo la diagonal con cero para que el mas parecido no sea si mismo
    numpy.fill_diagonal(matriz_ideal, 0)

    #obtener la posicion del mas cercano por fila
    posicion_min = numpy.argmin(matriz_distancias, axis=1)
    minimo = numpy.amin(matriz_distancias, axis=1)

    #posicion del mas similar por fila
    posicion_max_ideal = numpy.argmax(matriz_ideal, axis=1)

    #imprimir los documentos mas parecidos y su similitud
    filas = []
    total = matriz_distancias.shape[0]
    total_iguales = 0
    for i in range(total):
        nombre_documento = dataset_files[i].filename
        nombre_mas_similar = dataset_files[posicion_min[i]].filename
        distancia_menor = minimo[i]
        resultado = ""
        if posicion_min[i] == posicion_max_ideal[i]:
            resultado = "¡igual! :-)"
            total_iguales += 1
        else:
            nombre_correcto = dataset_files[posicion_max_ideal[i]].filename
            resultado = "¡mal! era {}".format(nombre_correcto)
        fila = {'id':  i,
                'query':  nombre_documento,
                'documento': nombre_mas_similar,
                'distancia coseno': distancia_menor,
                'resultado': resultado}
        filas.append(fila)

    print("resultados iguales al original: {} / {} = {:.1f}%".format(total_iguales, total, total_iguales/total*100))
    print()
    df = pandas.DataFrame(filas)
    print(df.to_string(index=False, justify='center'))

evaluar_mas_parecido(distancias, auto_similitud)

# Ejercicio Propuesto

Buscar documentos similares para textos de consulta libres.

In [ ]:
#Calcular el documento más similar para los siguientes documentos de consulta.
textos_q = [ "elecciones de alcalde", "estudiantes de universidad"]

#1.calcular la matriz de descriptores para textos_q (usar el vocabulario ya calculado)
#2.multiplicar los descriptores para obtener la similitud coseno
#2.buscar para cada descriptor de textos_q el más similar dentro de la matriz "descriptores"
#3.imprimir documentos más similares
